<a href="https://colab.research.google.com/github/JovitorzZ/ImageRec-CNN-HomerBart/blob/main/ImageRec_CNN_HomerBart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN para reconhecimento de imagens**

**Definição de Problema:**

A FOX deseja automatizar a identificação de personagens em cenas de episódios da série *Os Simpsons*, focando especificamente nos personagens Homer e Bart. Com milhares de episódios e uma vasta quantidade de cenas e frames, realizar esse processo manualmente é impraticável, demorado e propenso a erros humanos, o que limita a eficiência e a precisão na catalogação e análise dos conteúdos. Para resolver esse desafio, a FOX solicita ao seu time de tecnologia a implementação de uma rede neural convolucional (CNN) capaz de identificar automaticamente imagens dos personagens Homer e Bart.


---
**Objetivo do Projeto:**

Essa classificação automática permitirá a FOX:
- Criar conteúdo exclusivo focado nos personagens mais populares, potencializando oportunidades de merchandising e melhorando o engajamento do público.
- Oferecer uma experiência interativa e personalizada aos fãs, permitindo buscas avançadas e navegação por cenas específicas de seus personagens favoritos.
- Indexar e catalogar cenas de maneira automatizada, facilitando a criação de playlists temáticas e catálogos, como "momentos clássicos do Homer" ou "as maiores travessuras de Bart".


---
Ao implementar a CNN, a FOX visa não apenas otimizar processos internos e reduzir o tempo de trabalho, mas também expandir seu impacto no mercado de entretenimento ao proporcionar uma experiência de visualização inovadora e adaptada às preferências dos espectadores.

## 1° Passo:  Fazer o Download/Import das bibliotecas

Para este projeto iremos utilizar as versões:
* 2.16.1 do **TensorFlow**
* 1.26.4 do **Numpy**
* 5.5.0 do **Gradio**

In [ ]:
!pip install tensorflow==2.16.1 -q
!pip install numpy==1.26.4 -q
!pip install gradio==5.5.0 -q
#!pip install tempfile -q
#!pip install zipfile -q
#!pip install PIL -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.16.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
#Importando as bibliotecas
import numpy as np #Biblioteca para manipulação de arrays(vetores)
import tensorflow as tf #Biblioteca para rede neurais (Deep Learning)
import tempfile #Biblioteca para criar pastas temporarias
import zipfile  #Biblioteca para descompactar uma pasta zip
import gradio as gr #Biblioteca para criar uma interface grafica para o usuario
from PIL import Image  # Biblioteca Pillow para manipulação de imagem

In [ ]:
# Importações específicas para construir e treinar o modelo de rede neural
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

* **ImageDataGenerator**: Responsável por fazer a geração de novas imagens a partir das fotos iniciais e consequentimente aumentar o conjuto de dados.
* **Sequential**: Responsável por criar um modelo onde as camadas são empilhadas uma após a outra, como uma "pilha" linear de camadas.
* **Conv2D**: é uma camada de convolução que detecta características (como bordas) nas imagens.
* **MaxPooling2D**: Reduz o tamanho das imagens, mantendo apenas as informações principais (ajuda a simplificar o modelo).
* **Flatten**: Transforma as imagens em uma única linha de dados, preparando-as para as próximas camadas.
* **Dense**: é uma camada "densa" onde cada neurônio se conecta com todos os outros da camada seguinte, útil para tomar decisões.
* **Dropout**: "desativa" alguns neurônios aleatoriamente durante o treinamento, ajudando a evitar que o modelo memorize (overfitting).
* **BatchNormalization**: normaliza a saída de uma camada para ajudar a acelerar e estabilizar o treinamento.
* **InputLayer**: define o formato da entrada para o modelo, útil para especificar o tamanho da imagem de entrada.
* **Image**: Permite carregar e pré-processar imagens individuais para teste ou inferência, útil ao usar imagens fora do conjunto de dados principal.
* **EarlyStopping**: Interrompe o treinamento quando o modelo para de melhorar, evitando overfitting e economizando tempo.


## 2°Passo: Carregamento e Pré-processamento das imagens

### 2.1 Carregamento do dataset

In [ ]:
#Criando pasta temporaria
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory '/tmp/tmpxtlhqeyo'>


In [ ]:
#Lendo o arquivo dataset_personagens e extraindo para uma pasta temporária com o mesmo nome
with zipfile.ZipFile('/content/dataset_personagens.zip', 'r') as zip_ref: #Informar o caminho do seu dataset
  zip_ref.extractall(temp_dir.name)

### 2.2 Gerando novas imagens a partir das que já contém no dataset

In [ ]:
gerador_treinamento = ImageDataGenerator(rescale = 1/255, #Normalizando os valores do pixels das imagens
                                         rotation_range = 360, #Rotação aleatória das imagens no intervalo de -360 a +360 graus
                                         width_shift_range = 0.3, #Deslocamento aleatório horizontal
                                         height_shift_range = 0.3, #Deslocamento aleatório vertical
                                         brightness_range = (0.1, 0.9), #Alteração aleatória em um range de 10% até 90% da luminosidade
                                         horizontal_flip = True, #Realiza a inversao horizontal, espelhamento da imagem
                                         vertical_flip = True, #Realiza a inversao vertical, espelhamento da imagem
                                         shear_range = 0.2, #realiza o deslocamento da imagem criando distorcao
                                         zoom_range = 0.5 #aplica zoom
)

Salvando a base de treino criada apartir das novas imagens na pasta temporária: /dataset_personagens/training_set

Com seu tamanho sendo 64x64 pixels e proceesando as em blocos de 32 imagens, por ser somente imagens do Bart e do Homer será uma classificação Binaria: 1 para o **Homer** e 0 para o **Bart**.


In [ ]:
base_treino = gerador_treinamento.flow_from_directory(f'{temp_dir.name}/dataset_personagens/training_set',
                                                target_size = (64,64),
                                                batch_size = 32,
                                                class_mode = 'binary' #Se fosse um problema onde serão identificado mais personagens o class_mode deve ser trocado para 'categorical'
                                                )

Found 196 images belonging to 2 classes.


In [ ]:
#Normalizando os valores do pixels das imagens de teste
gerador_teste = ImageDataGenerator(rescale = 1/255)

In [ ]:
#Executando o mesmo processo da base_treino na base_teste
base_teste = gerador_teste.flow_from_directory(f'{temp_dir.name}/dataset_personagens/test_set',
                                                target_size = (64,64),
                                                batch_size = 32,
                                                class_mode = 'binary'
                                                )

Found 73 images belonging to 2 classes.


## 3°Passo: Treinamento da Rede Neural Convolucional

**CNN sequencial**: É uma arquitetura de rede neural onde as camadas são empilhadas de forma linear, uma após a outra.

In [ ]:
cnn = Sequential()

#Definido camada de entrada do modelo
cnn.add(InputLayer(input_shape = (64,64,3)))  #Espera imagens de tamanho 64x64 com 3 canais (RGB)

#Primeira camada de convolução
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu')) #Aplica 32 filtros de tamanho 3x3 à imagem de entrada, utilizando a função de ativação ReLU
cnn.add(BatchNormalization()) #Normaliza a saída da camada anterior para acelerar o treinamento
cnn.add(MaxPooling2D(pool_size = (2,2))) #Reduz pela metade a resolução, mantendo as características principais

#Segunda camada de convolução, semelhante à primeira, para aprofundar a extração de características
#Troquei a função de ativação para Laeky_relu para ajudar a evitar o problema da "morte dos neurônios" que pode ocorrer com ReLU, permitindo que pequenos valores negativos passem
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'leaky_relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size = (2,2)))

#Achata a saída das camadas anteriores (matriz 2D) em um vetor 1D
cnn.add(Flatten())

#Camada Densa
cnn.add(Dense(units = 128, activation = 'relu'))#Com 128 unidades e função de ativação ReLU
cnn.add(Dropout(0.3)) #Dropout para prevenir overfitting, desativando aleatoriamente 30% dos neurônios durante o treinamento

#Camada de saída com um único neurônio e função de ativação sigmoide para classificação binária
cnn.add(Dense(units = 1, activation = 'sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 29, 29, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 813,473 (3.10 MB)

 Trainable params: 813,345 (3.10 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
# Configurando o Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',         #Monitorando a perda na validação
    patience=10,                #Número de épocas sem melhora para interromper o treinamento
    verbose=1,                  #Imprime mensagens de parada antecipada
    restore_best_weights=True   #Restaura os pesos da melhor época
)

In [ ]:
cnn.compile( loss = 'binary_crossentropy', metrics = ['accuracy'])

O código compila o modelo de rede neural convolucional (CNN) usando o otimizador especificado, a função de perda "binary_crossentropy" (adequada para problemas de classificação binária) e a métrica "accuracy" para avaliar o desempenho durante o treinamento.

In [ ]:
cnn.fit(base_treino, epochs=100, validation_data=base_teste, callbacks=[early_stopping])

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 359ms/step - accuracy: 0.4852 - loss: 4.1155 - val_accuracy: 0.5753 - val_loss: 1.3210
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 320ms/step - accuracy: 0.5460 - loss: 2.5421 - val_accuracy: 0.7123 - val_loss: 0.6617
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 176ms/step - accuracy: 0.6214 - loss: 1.3881 - val_accuracy: 0.5068 - val_loss: 0.6652
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.6402 - loss: 1.0411 - val_accuracy: 0.6712 - val_loss: 0.6199
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 0.6575 - loss: 0.7260 - val_accuracy: 0.6575 - val_loss: 0.6360
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - accuracy: 0.6545 - loss: 0.8501 - val_accuracy: 0.6301 - val_loss: 0.6791
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 336ms/step - accuracy: 0.6781 - loss: 0.7105 - val_accuracy: 0.7397 - val_loss: 0.6395
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.6147 - loss: 0.8157 - val_accuracy: 0.4247 - val_loss: 0.7

## 3.1 Resultado de treinamento da rede neural apresenta as seguintes métricas:



- **accuracy: 0.7140**: A precisão (accuracy) do modelo no conjunto de treinamento é de aproximadamente 71,40%, indicando que o modelo classificou corretamente cerca de 71% dos exemplos de treinamento.
  
- **loss: 0.5553**: A função de perda (loss) no conjunto de treinamento é de 0,5553. Esse valor representa o erro do modelo, sugere que o modelo ainda comete alguns erros no treinamento, o que é comum e esperado, pois um valor muito próximo de zero poderia indicar overfitting

- **val_accuracy: 0.8082**: A precisão do modelo no conjunto de validação é de 80,82%, o que é superior à precisão do treinamento. Isso sugere que o modelo está generalizando bem para dados não vistos.

- **val_loss: 0.5151**: A perda no conjunto de validação é de 0,5151 levemente inferior à perda no conjunto de treinamento. Isso indica que o modelo está conseguindo manter um bom desempenho sem sinais claros de overfitting (ajuste excessivo aos dados de treinamento).

Esses resultados mostram que o modelo está com uma performance aceitável e que há espaço para melhorar, mas já possui uma boa capacidade de generalização para dados novos.

## 4°Passo: Prevendo novas imagens

In [ ]:
base_treino.class_indices

{'bart': 0, 'homer': 1}

In [ ]:
def predict_image_interface(uploaded_image):
    #Converte a imagem carregada para um formato compatível
    imagem_teste = Image.fromarray(np.array(uploaded_image))  #Garante que a imagem é compatível com Pillow
    imagem_teste = imagem_teste.resize((64, 64))  #Redimensiona para 64x64

    #Converte a imagem redimensionada em um array
    imagem_teste = image.img_to_array(imagem_teste)  #Converte para array NumPy
    imagem_teste /= 255  #Padroniza para valores entre [0, 1]
    imagem_teste = np.expand_dims(imagem_teste, axis=0)  #Expande para (1, 64, 64, 3)

    #Faz a previsão
    previsao = cnn.predict(imagem_teste)[0]

    #Retorna o resultado da previsão
    return "Bart" if previsao < 0.5 else "Homer"

#Configurando a interface do Gradio
interface = gr.Interface(
    fn=predict_image_interface,
    inputs="image",
    outputs="text",
    title="Classificador Homer e Bart",
    description="Envie uma imagem para classificar se é Homer ou Bart"
)

interface.launch(debug=True)  #Ativa o modo debug para exibir erros detalhados


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e97b5ca71d4c440e64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e97b5ca71d4c440e64.gradio.live


# Conclusão


O modelo apresenta uma boa capacidade de generalização, mas ainda classifica incorretamente algumas imagens quando testado com novos dados. Isso pode indicar que o modelo não capturou completamente todas as variações dos padrões visuais nas classes. A adição de mais dados ou ajustes no treinamento podem ajudar a refinar essa precisão.

**Próximos Passos**:

1. **Analisar e Identificar Padrões de Erro**:
    - Avaliar uma amostra das imagens classificadas incorretamente para verificar se há similaridades ou padrões nas classes confundidas.

2. **Expandir e Diversificar os Dados de Treinamento**:
   - Se possível, incluir mais imagens representativas das classes que foram classificadas incorretamente.

3. **Realizar Fine-Tuning**:
   - Retreinar o modelo com as novas imagens ou com um conjunto de dados atualizado pode ajudar a melhorar o reconhecimento em situações previamente problemáticas.

4. **Avaliar com Métricas Detalhadas**:
   - Gerar uma **matriz de confusão** para ver quais classes são mais frequentemente confundidas. Também analisar precisão, recall e F1-score para ter uma visão mais detalhada do desempenho.
